# Week4 (Wuwei Zhang)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mnist import MNIST

def load_dataset():
    mndata = MNIST('C:/Users/Zhang/python-mnist/data')
    A_train, labels_train = map(np.array, mndata.load_training()) 
    A_test, labels_test = map(np.array, mndata.load_testing()) 
    A_train = A_train/255.0
    A_test = A_test/255.0
    
    return A_train, A_test, labels_train, labels_test

A_train, A_test, labels_train, labels_test = load_dataset()


B_labels_train = np.zeros((60000, 10))
for i in range(len(labels_train)):
    B_labels_train[i, labels_train[i]] = 1
B_labels_test = np.zeros((10000, 10))
for j in range(len(labels_test)):
    B_labels_test[j, labels_test[j]] = 1
    


# Create Binary Classification Dataset   

ind2 = np.where(labels_train == 2)[0]
ind7 = np.where(labels_train ==7)[0]

bin_A_train = np.vstack((A_train[ind2,:], A_train[ind7,:]))
bin_label_train = np.hstack((-1*np.ones(len(ind2)), np.ones(len(ind7))))


ind2_test = np.where(labels_test == 2)[0]
ind7_test = np.where(labels_test ==7)[0]

bin_A_test = np.vstack((A_test[ind2_test,:], A_test[ind7_test,:]))
bin_label_test = np.hstack((-1*np.ones(len(ind2_test)), np.ones(len(ind7_test))))



In [ ]:
# %% Minimize J(x,y) using gradient descent

# Define err = $\eta||x_{t+1} - x_t||_2 + \eta|y_{t+1} - y|$

def mu_i(x, y, i):
    return 1/(1 + np.exp((-1)*bin_label_train[i]*(y+bin_A_train[i,]@x)))

              
def J_grad_x(x, y):
    s = np.zeros(784)
    for i in range(12223):
        s1 = (-1)*bin_label_train[i]*(bin_A_train[i,])*(1-mu_i(x, y, i))
        s = np.add(s, s1)
    return s / 12223 + 0.01 * 2 * x


def J_grad_y(x, y):
    s = 0
    for i in range(12223):
        s1 = (-1)*bin_label_train[i]*(1-mu_i(x, y, i))
        s = np.add(s, s1)
    return s / 12223


def J(x, y):
    s = 0;
    for i in range(12223):
        s1 = 1 + np.exp((-1)*bin_label_train[i]*(y+bin_A_train[i,]@x))
        s = s + np.log(s1)
    return s / 12223 + 0.01 * np.square(x).sum()/784


def J_testing(x, y):
    s = 0;
    for i in range(2060):
        s1 = 1 + np.exp((-1)*bin_label_test[i]*(y+bin_A_test[i,]@x))
        s = s + np.log(s1)
    return s / 2060 + 0.01 * np.square(x).sum()/784


def grad_descent(xinit, yinit, J_grad_x, J_grad_y, **kwargs):
    tol = kwargs.pop('error', 1e-6)
    step_size = kwargs.pop('step_size', .1)
    
    err = 1
    xold = xinit
    yold = yinit
    
    max_iter = 500
    iter_count = 0
    
    J_value = [J(xold, yold)]
    J_value_testing = [J_testing(xold, yold)]
    
    while err > tol:
        xnew = xold - step_size*J_grad_x(xold, yold)
        ynew = yold - step_size*J_grad_y(xnew, yold)
        
        err1 = step_size * np.linalg.norm(xnew - xold,2)
        err2 = step_size * abs(ynew - yold)
        err = err1+err2
        
        J_value.append(J(xnew, ynew))
        J_value_testing.append(J_testing(xnew, ynew))
        
        iter_count+=1
        if iter_count>=max_iter:
            print('Reached max number of iterations')
            return xnew, ynew, J_value, J_value_testing
        
    return xnew, ynew, J_value, J_value_testing
        
    
    
    
xfinal, yfinal, J_value_train, J_value_testing = grad_descent(np.zeros(784).T, 0, 
                                                 J_grad_x, J_grad_y, step_size = 1e-3) 


plt.plot(range(501), J_value_train, label="J value for training") 
plt.plot(range(501), J_value_testing, label="J value for testing") 
plt.xlabel('J value')
plt.ylabel('Iteration')
plt.legend()
plt.show()




# Computer Classification Error

def clasError(bin_A, bin_label, num):
    countError = 0
    for i in range(num):
        predict = bin_label[i] + bin_A[i, ]@xfinal
        if (predict < 0 and bin_label[i] > 0):
            countError = countError + 1
        elif (predict > 0 and bin_label[i] < 0):
            countError = countError + 1
            
    return countError/num
    
    
print(clasError(bin_A_train, bin_label_train, 12223))    